# Data ingestion to cloud storage
1. Deploy a spark cluster (Dataproc, EMR, Azure HDinsight)
2. Store data in HDFS and not local
3. Use Pyspark to interact with data
So we will do the whole process in dataproc cluster
4. Start a cluster in dataproc
5. open the SSH
6. create a new directory for your data
   `mkdir olist`
7. change your working directory to this new one
   `cd olist`
8. load the data you want in there, we can do this by pasting the cURL command here
```
#!/bin/bash
curl -L -o ~/olist/brazilian-ecommerce.zip\
  https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce
```
9. Create another directory for your unzipped data storage
10. unzip the file
    `unzip file_name -d destination_path`
So we successfully obtained the data into our local or master, now we have to migrate to hdfs
```bash
-- To create a directory for our data
hdfs dfs -mkdir /data/olist/


-- To put all the data from local to hdfs
hadoop fs -put ~/olist/data/*.csv /data/olist/
```


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName('Olist')\
.getOrCreate()

25/08/22 05:33:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark

In [4]:
!hadoop fs -ls /data/olist

Found 9 items
-rw-r--r--   2 sibashishsatipali hadoop    9033957 2025-08-21 20:26 /data/olist/olist_customers_dataset.csv
-rw-r--r--   2 sibashishsatipali hadoop   61273883 2025-08-21 20:26 /data/olist/olist_geolocation_dataset.csv
-rw-r--r--   2 sibashishsatipali hadoop   15438671 2025-08-21 20:26 /data/olist/olist_order_items_dataset.csv
-rw-r--r--   2 sibashishsatipali hadoop    5777138 2025-08-21 20:26 /data/olist/olist_order_payments_dataset.csv
-rw-r--r--   2 sibashishsatipali hadoop   14451670 2025-08-21 20:26 /data/olist/olist_order_reviews_dataset.csv
-rw-r--r--   2 sibashishsatipali hadoop   17654914 2025-08-21 20:26 /data/olist/olist_orders_dataset.csv
-rw-r--r--   2 sibashishsatipali hadoop    2379446 2025-08-21 20:26 /data/olist/olist_products_dataset.csv
-rw-r--r--   2 sibashishsatipali hadoop     174703 2025-08-21 20:26 /data/olist/olist_sellers_dataset.csv
-rw-r--r--   2 sibashishsatipali hadoop       2613 2025-08-21 20:26 /data/olist/product_category_name_translation.c

In [5]:
#path for our data
hdfs_path = '/data/olist/'

In [6]:
customer_df = spark.read.csv(hdfs_path+'olist_customers_dataset.csv', header = True, inferSchema =True)
geolocation_df = spark.read.csv(hdfs_path+'olist_geolocation_dataset.csv', header = True, inferSchema =True)
order_items_df = spark.read.csv(hdfs_path+'olist_order_items_dataset.csv', header = True, inferSchema =True)
order_payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv' ,header = True, inferSchema = True)
order_reviews_df = spark.read.csv(hdfs_path+'olist_order_reviews_dataset.csv', header = True, inferSchema =True)
orders_df = spark.read.csv(hdfs_path+'olist_orders_dataset.csv', header = True, inferSchema =True)
products_df = spark.read.csv(hdfs_path+'olist_products_dataset.csv', header = True, inferSchema =True)
sellers_df = spark.read.csv(hdfs_path+'olist_sellers_dataset.csv', header = True, inferSchema =True)
category_name_df = spark.read.csv(hdfs_path+'product_category_name_translation.csv', header = True, inferSchema =True)

In [7]:
customer_df.show(10)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [8]:
customer_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [9]:
#data leakageor drop
print(f'customers : {customer_df.count()} rows')

customers : 99441 rows


From above value we can match the obtained dta from the source to check for any leakage

In [10]:
print(f'orders : {orders_df.count()} rows')

orders : 99441 rows


We obtained there is no data leakage 

## Checking for Null values

In [11]:
#checking for null and duplicate values
from pyspark.sql.functions import col
customer_df.select([col(c).isNull().alias(c) for c in customer_df.columns]).show()

+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|           

In [14]:
from pyspark.sql.functions import col,when,count
customer_df.select([count(when(col(c).isNull(),1)).alias(c) for c in customer_df.columns]).show()

+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



In [15]:
geolocation_df.select([count(when(col(c).isNull(),1)).alias(c) for c in geolocation_df.columns]).show()

+---------------------------+---------------+---------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+---------------+---------------+----------------+-----------------+
|                          0|              0|              0|               0|                0|
+---------------------------+---------------+---------------+----------------+-----------------+



In [16]:
order_items_df.select([count(when(col(c).isNull(),1)).alias(c) for c in order_items_df.columns]).show()

+--------+-------------+----------+---------+-------------------+-----+-------------+
|order_id|order_item_id|product_id|seller_id|shipping_limit_date|price|freight_value|
+--------+-------------+----------+---------+-------------------+-----+-------------+
|       0|            0|         0|        0|                  0|    0|            0|
+--------+-------------+----------+---------+-------------------+-----+-------------+



In [18]:
order_payments_df.select([count(when(col(c).isNull(),1)).alias(c) for c in order_payments_df.columns]).show()

+--------+------------------+------------+--------------------+-------------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------+------------------+------------+--------------------+-------------+
|       0|                 0|           0|                   0|            0|
+--------+------------------+------------+--------------------+-------------+



In [19]:
order_reviews_df.select([count(when(col(c).isNull(),1)).alias(c) for c in order_reviews_df.columns]).show()

+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|review_id|order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|        1|    2236|        2380|               92157|                 63079|                8764|                   8785|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+



In [20]:
orders_df.select([count(when(col(c).isNull(),1)).alias(c) for c in orders_df.columns]).show()

+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



In [21]:
products_df.select([count(when(col(c).isNull(),1)).alias(c) for c in products_df.columns]).show()

+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|         0|                  610|                610|                       610|               610|               2|                2|                2|               2|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+



In [22]:
sellers_df.select([count(when(col(c).isNull(),1)).alias(c) for c in sellers_df.columns]).show()

+---------+----------------------+-----------+------------+
|seller_id|seller_zip_code_prefix|seller_city|seller_state|
+---------+----------------------+-----------+------------+
|        0|                     0|          0|           0|
+---------+----------------------+-----------+------------+



In [23]:
category_name_df.select([count(when(col(c).isNull(),1)).alias(c) for c in category_name_df.columns]).show()

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|                    0|                            0|
+---------------------+-----------------------------+



So form this analysis we got that there are many tables with null values in there columns

## Checking for duplicate values

In company by looking through the dataset we can find the unique_column

In [27]:
customer_df.groupBy('customer_id').count().filter('count>1').show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



In [29]:
sellers_df.groupBy('seller_id').count().filter('count>1').show()

+---------+-----+
|seller_id|count|
+---------+-----+
+---------+-----+



In [30]:
products_df.groupBy('product_id').count().filter('count>1').show()

+----------+-----+
|product_id|count|
+----------+-----+
+----------+-----+



## Business level exploration

In [32]:
# Customer distribution by state
customer_df.groupBy('customer_state').count().orderBy('count', ascending = False).show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [34]:
#order status distribution
orders_df.groupBy('order_status').count().orderBy('count', ascending = False).show()

+------------+-----+
|order_status|count|
+------------+-----+
|   delivered|96478|
|     shipped| 1107|
|    canceled|  625|
| unavailable|  609|
|    invoiced|  314|
|  processing|  301|
|     created|    5|
|    approved|    2|
+------------+-----+



In [35]:
#payment status distribution
order_payments_df.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [36]:
order_payments_df.groupBy('payment_type').count().orderBy('count', ascending = False).show()

+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+



In [37]:
# top selling items
order_items_df.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [39]:
from pyspark.sql.functions import sum

product_sales = order_items_df.groupBy('product_id').agg(sum('price').alias('total_sales'))
product_sales.orderBy('total_sales', ascending = False).show()

+--------------------+------------------+
|          product_id|       total_sales|
+--------------------+------------------+
|bb50f2e236e5eea01...|           63885.0|
|6cdd53843498f9289...| 54730.20000000005|
|d6160fb7873f18409...|48899.340000000004|
|d1c427060a0f73f6b...| 47214.51000000006|
|99a4788cb24856965...|43025.560000000085|
|3dd2a17168ec895c7...| 41082.60000000005|
|25c38557cf793876c...| 38907.32000000001|
|5f504b3a1c75b73d6...|37733.899999999994|
|53b36df67ebb7c415...| 37683.42000000001|
|aca2eb7d00ea1a7b8...| 37608.90000000007|
|e0d64dcfaa3b6db5c...|          31786.82|
|d285360f29ac7fd97...|31623.809999999983|
|7a10781637204d8d1...|           30467.5|
|f1c7f353075ce59d8...|          29997.36|
|f819f0c84a64f02d3...|29024.479999999996|
|588531f8ec37e7d5f...|28291.989999999998|
|422879e10f4668299...|26577.219999999972|
|16c4e87b98a9370a9...|           25034.0|
|5a848e4ab52fd5445...|24229.029999999962|
|a62e25e09e05e6faf...|           24051.0|
+--------------------+------------

In [42]:
# average delivery time analysis
delivery_df = orders_df.select('order_id', 'order_purchase_timestamp', 'order_delivered_customer_date')


In [43]:
delivery_df.show()

+--------------------+------------------------+-----------------------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|
+--------------------+------------------------+-----------------------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|
|a4591c265e18cb1dc...|     2017-07-09 21:57:05|          2017-07-26 10:57:55|
|136cce7faa42fdb2c...|     2017-04-11 12:22:08|                         NULL|
|6514b8ad8028c9f2c...|     2017-05-16 13:10:30|          2017-05-26 12:55:51|
|76c6e866289321a7c...|     2017-01-23 18:29:09|          2017-02-02 14:08:10|
|e69bfb5eb88e0ed6a...|     2017-07-29 11:55:02|          2017-08

In [44]:
from pyspark.sql.functions import datediff, to_date
delivery_details_df = delivery_df.withColumn('delivery_time', datediff(col('order_delivered_customer_date'),col('order_purchase_timestamp')))

In [46]:
delivery_details_df.orderBy('delivery_time', ascending = False).show()

+--------------------+------------------------+-----------------------------+-------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|delivery_time|
+--------------------+------------------------+-----------------------------+-------------+
|ca07593549f1816d2...|     2017-02-21 23:31:27|          2017-09-19 14:36:39|          210|
|1b3190b2dfa9d789e...|     2018-02-23 14:57:35|          2018-09-19 23:24:07|          208|
|440d0d17af552815d...|     2017-03-07 23:59:51|          2017-09-19 15:12:50|          196|
|2fb597c2f772eca01...|     2017-03-08 18:09:02|          2017-09-19 14:33:17|          195|
|285ab9426d6982034...|     2017-03-08 22:47:40|          2017-09-19 14:00:04|          195|
|0f4519c5f1c541dde...|     2017-03-09 13:26:57|          2017-09-19 14:38:21|          194|
|47b40429ed8cce3ae...|     2018-01-03 09:44:01|          2018-07-13 20:51:31|          191|
|2fe324febf907e3ea...|     2017-03-13 20:17:10|          2017-09-19 17:00:07|   